In [2]:
#!gsutil -m cp -r gs://neurips-adc-bucket/raw/train/100468857 ../data/raw/train
#!gsutil -m cp -r gs://neurips-adc-bucket/raw/train/1005054328 ../data/raw/train
#!gsutil cp gs://neurips-adc-bucket/raw/* ../data/raw/

In [1]:
from google.cloud import storage
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ibeam
import os
import subprocess
import glob
import shutil
from typing import Union
from pathlib import Path
import pandas as pd
import numpy as np
from collections import defaultdict
from typing import List, Tuple, Any
from astropy.stats import sigma_clip
import itertools

In [4]:
storage_client = storage.Client()

In [5]:
blobs = storage_client.list_blobs(
    bucket_or_name="neurips-adc-bucket", match_glob="raw/**/*", delimiter="/"
)

In [6]:
[blob for blob in blobs.prefixes]

[]

In [7]:
def list_blobs(bucket_name):
    """Lists all the blobs in the bucket."""
    # bucket_name = "your-bucket-name"

    storage_client = storage.Client()

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name)

    # Note: The call returns a response only when the iterator is consumed.
    for blob in blobs:
        print(blob.name)

In [8]:
def copy_to(element, dest: str):
    sufix = "/".join(element.split("/")[-2:])
    output = os.path.join(dest, sufix)
    shutil.copytree(src=element, dst=output)

In [10]:
with beam.Pipeline() as pipeline:
    outputs = (
        pipeline
        | "Create initial values" >> beam.Create(glob.glob("../data/raw/**/*"))
        | "Copy the folder" >> beam.Map(lambda x: copy_to(x, "../data/output"))
    )
    outputs | beam.Map(print)

None
None


Next setp appliquer la transformation de bout en bout dans le notebook.
quand ce sera bon on va voir comment intéger cela dans un ExampleGen

# LS FUNCTION

In [11]:
from typing import Union
from pathlib import Path

In [12]:
type(os.path.join(".", "data"))

str

In [11]:
def ls_gcp_bucket(args: list[str]) -> list[str]:
    """List providers, buckets, or objects

    Args:
        command (list[str]): args and options supproted by `gsutil ls`

    Returns:
        list[str]: Informations returned by `gsutil ls` command as list for each line.
    """
    command = ["gsutil", "ls"] + args
    result = subprocess.run(args=command, capture_output=True, text=True)
    if result.returncode != 0:
        # TODO: logging
        print(f"{result.stderr}")
    else:
        output = result.stdout.splitlines()
        return output

In [12]:
os.listdir()

['02-host-update-calibrating-and-binning-astronomical-data.ipynb',
 '03-host-starter-solution.ipynb',
 '01-cay-understanding-the-data.ipynb',
 '.gitkeep',
 '04-cay-test-apache-beam-tfx.ipynb']

In [19]:
def ls_filesystem(
    path: Union[str, Path], files_only: bool = False, folders_only: bool = False
) -> list[str]:
    """List files in path.

    Args:
        path (Union[str, Path]): The path whose objects to list.
        files_only (bool, optional): Whether to return files only. Defaults to False.
        folders_only (bool, optional): Whether to return folders only. Defaults to False.

    Returns:
        list[str]: list of element in path.
    """
    if files_only and folders_only:
        folders_only = False
    ls = [os.path.join(path, item) for item in os.listdir(path)]
    if files_only:
        files = []
        for item in ls:
            if os.path.isfile(item):
                files.append(item)
        ls = files
    if folders_only:
        folders = []
        for item in ls:
            if os.path.isdir(item):
                folders.append(item)
        ls = folders
    return ls

In [20]:
class ProcessingDoFn(beam.DoFn):
    def __init__(self, axis_info, train_adc, test_adc):
        self.axis = axis_info
        self.adc = train_adc
        self.tadc = test_adc

    def process(self, element):
        return [self.axis, self.adc, self.tadc, element]

In [21]:
class ADCRevertFn:
    def process(self, element):
        airs_files = {}
        fgs_files = {}
        for item in os.listdir(element):
            if "airs" in item.lower():
                if "signal" in item.lower():
                    airs_files["signal"] = pd.read_parquet(path=item)
                if os.path.isdir(item):
                    for file in os.listdir(item):
                        if "dark" in file:
                            airs_files["dark"] = pd.read_parquet(
                                os.path.join(item, file)
                            )
                        if "dead" in file:
                            airs_files["dead"] = pd.read_parquet(
                                os.path.join(item, file)
                            )
                        if "flat" in file:
                            airs_files["flat"] = pd.read_parquet(
                                os.path.join(item, file)
                            )
                        if "read" in file:
                            airs_files["read"] = pd.read_parquet(
                                os.path.join(item, file)
                            )
            if "fgs" in item.lower():
                if "signal" in item.lower():
                    fgs_files["signal"] = pd.read_parquet(path=item)
                if os.path.isdir(item):
                    for file in os.listdir(item):
                        if "dark" in file:
                            fgs_files["dark"] = pd.read_parquet(
                                os.path.join(item, file)
                            )
                        if "dead" in file:
                            fgs_files["dead"] = pd.read_parquet(
                                os.path.join(item, file)
                            )
                        if "flat" in file:
                            fgs_files["flat"] = pd.read_parquet(
                                os.path.join(item, file)
                            )
                        if "read" in file:
                            fgs_files["read"] = pd.read_parquet(
                                os.path.join(item, file)
                            )
        airs_files["signal"] = (
            fgs_files["signal"]
            .astype(np.float64)
            .reshape((fgs_files["signal"].shape[0], 32, 356))
        )
        airs_files["signal"]
        files = os.listdir(element)
        signal = signal.astype(np.float64)
        signal /= gain
        signal += offset
        return signal

In [22]:
l = ls_gcp_bucket(["gs://neurips-adc-bucket/**"])

In [23]:
l

['gs://neurips-adc-bucket/raw/',
 'gs://neurips-adc-bucket/raw/axis_info.parquet',
 'gs://neurips-adc-bucket/raw/sample_submission.csv',
 'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/dark.parquet',
 'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/dead.parquet',
 'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/flat.parquet',
 'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/linear_corr.parquet',
 'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/read.parquet',
 'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_signal.parquet',
 'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/dark.parquet',
 'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/dead.parquet',
 'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/flat.parquet',
 'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/linear_corr.parquet',
 'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/rea

In [39]:
"gs://neurips-adc-bucket/raw/train/1468955155/FGS1_signal.parquet".split("/")[5]

'1468955155'

In [54]:
def split_key_value(path: str):
    l = path.split("/")
    if len(l) <= 5:
        return (l[3], "/".join(l) if l[4] != "" else None)
    else:
        return (l[5], "/".join(l))

In [34]:
l[0]

'gs://neurips-adc-bucket/raw/'

In [51]:
len("gs://neurips-adc-bucket/raw/".split("/"))

5

In [55]:
[split_key_value(item) for item in l]

[('raw', None),
 ('raw', 'gs://neurips-adc-bucket/raw/axis_info.parquet'),
 ('raw', 'gs://neurips-adc-bucket/raw/sample_submission.csv'),
 ('499191466',
  'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/dark.parquet'),
 ('499191466',
  'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/dead.parquet'),
 ('499191466',
  'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/flat.parquet'),
 ('499191466',
  'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/linear_corr.parquet'),
 ('499191466',
  'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/read.parquet'),
 ('499191466',
  'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_signal.parquet'),
 ('499191466',
  'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/dark.parquet'),
 ('499191466',
  'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/dead.parquet'),
 ('499191466',
  'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/flat.parque

In [28]:
"gs://neurips-adc-bucket/raw/".split("/")

['gs:', '', 'neurips-adc-bucket', 'raw', '']

In [27]:
with beam.Pipeline() as pipeline:
    train_adc = pd.read_csv("../data/raw/train_adc_info.csv")
    test_adc = pd.read_csv("../data/raw/test_adc_info.csv")
    axis = pd.read_parquet("../data/raw/axis_info.parquet")
    outputs = (
        pipeline
        | "Create initial values"
        >> beam.Create(ls_gcp_bucket(["gs://neurips-adc-bucket/**"]))
        | "Ls filesystem" >> beam.Map(lambda x: copy_to(x, "../data/output"))
    )
    outputs | beam.Map(print)

ERROR:apache_beam.runners.common:FileNotFoundError: [Errno 2] No such file or directory: 'gs://neurips-adc-bucket/raw/' [while running '[27]: Ls filesystem']
Traceback (most recent call last):
  File "apache_beam/runners/common.py", line 1495, in apache_beam.runners.common.DoFnRunner.process
  File "apache_beam/runners/common.py", line 688, in apache_beam.runners.common.SimpleInvoker.invoke_process
  File "/home/drxc/.pyenv/versions/3.10.13/envs/neurips/lib/python3.10/site-packages/apache_beam/transforms/core.py", line 2065, in <lambda>
    wrapper = lambda x: [fn(x)]
  File "/tmp/ipykernel_27823/1931494985.py", line 9, in <lambda>
    | "Ls filesystem" >> beam.Map(lambda x: copy_to(x, "../data/output"))
  File "/tmp/ipykernel_27823/1526363044.py", line 4, in copy_to
    shutil.copytree(src=element, dst=output)
  File "/home/drxc/.pyenv/versions/3.10.13/lib/python3.10/shutil.py", line 557, in copytree
    with os.scandir(src) as itr:
FileNotFoundError: [Errno 2] No such file or directo

RuntimeError: FileNotFoundError: [Errno 2] No such file or directory: 'gs://neurips-adc-bucket/raw/' [while running '[27]: Ls filesystem']

In [28]:
class ProcessingDoFn(beam.DoFn):
    def __init__(self, axis_info, train_adc, test_adc):
        self.axis = axis_info
        self.adc = train_adc
        self.tadc = test_adc

    def process(self, element):
        return [self.axis, self.adc, self.tadc, element]

In [57]:
class ProcessingDoFn(beam.DoFn):
    def process(self, element):
        return [element]

In [64]:
paths

[('499191466',
  'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/dark.parquet'),
 ('499191466',
  'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/dead.parquet'),
 ('499191466',
  'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/flat.parquet'),
 ('499191466',
  'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/linear_corr.parquet'),
 ('499191466',
  'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/read.parquet'),
 ('499191466',
  'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_signal.parquet'),
 ('499191466',
  'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/dark.parquet'),
 ('499191466',
  'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/dead.parquet'),
 ('499191466',
  'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/flat.parquet'),
 ('499191466',
  'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/linear_corr.parquet'),
 ('499191466',
  'gs://neurips-a

In [90]:
def group_by_key(key_value: List[Tuple[Any, Any]]) -> List[List[Any]]:
    """
    Group a list of key-value pairs by key.

    Args:
        key_value (List[Tuple[Any, Any]]): List of tuples where each tuple contains a key and a value.

    Returns:
        List[List[Any]]: A list of lists where each sublist contains the key and a list of associated values.
    """
    grouped_data = defaultdict(list)
    for key, value in key_value:
        grouped_data[key].append(value)
    return [[key, values] for key, values in grouped_data.items()]

In [91]:
def get_raw_data_uris(pattern: str) -> List[Tuple[str, List[str]]]:
    """
    Get the data URIs in the Google Cloud Storage bucket.

    Args:
        pattern (str): Glob pattern like in gsutil ls.

    Returns:
        List[Tuple[str, List[str]]]: List of tuples where each tuple has the planet ID as the key and its data as the value.
    """
    paths = ls_gcp_bucket([pattern])
    key_value_pairs = [split_key_value(path=path) for path in paths]
    raw_data = [
        item[1] for item in key_value_pairs if item[0] == "raw" and item[1] is not None
    ]
    non_raw_data = [item for item in key_value_pairs if item[0] != "raw"]
    grouped_data = group_by_key(non_raw_data)
    result = [(key, value + raw_data) for key, value in grouped_data]
    return result

In [92]:
get_raw_data_uris("gs://neurips-adc-bucket/**")

[('499191466',
  ['gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/dark.parquet',
   'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/dead.parquet',
   'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/flat.parquet',
   'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/linear_corr.parquet',
   'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/read.parquet',
   'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_signal.parquet',
   'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/dark.parquet',
   'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/dead.parquet',
   'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/flat.parquet',
   'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/linear_corr.parquet',
   'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/read.parquet',
   'gs://neurips-adc-bucket/raw/test/499191466/FGS1_signal.parquet',
   'gs://neurips-ad

In [ ]:
UnicodeDecodeError

In [ ]:
ArrowInvalide

In [95]:
df = pd.read_parquet("../data/raw/test_adc_info.csv")

ArrowInvalid: Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

In [94]:
df = pd.read_csv("../data/raw/train/100468857/FGS1_signal.parquet")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc0 in position 7: invalid start byte

In [10]:
class CalibrationFn(beam.DoFn):
    def __init__(
        self,
        cut_inf: int,
        cut_sup: int,
        binning: int,
        mask: bool,
        corr: bool,
        dark: bool,
        flat: bool,
    ):
        self.CUT_INF = cut_inf
        self.CUT_SUP = cut_sup
        self.BINNING = binning
        self.MASK = mask
        self.CORR = corr
        self.DARK = dark
        self.FLAT = flat

    def process(self, element):
        id, uris = element
        airs_data, fgs_data, info_data = self._load_data(uris)
        airs_signal = self._calibrate_airs_data(id, airs_data, info_data)
        fgs_signal = self._calibrate_airs_data(id, fgs_data, info_data)
        return [id, airs_signal, fgs_signal]

    def _calibrate_airs_data(
        self, id: str, data: dict[pd.DataFrame], info: dict[pd.DataFrame]
    ):
        signal = data["signal"]
        gain = info["adc_info"].set_index("planet_id")["AIRS-CH0_adc_gain"].loc[int(id)]
        offset = (
            info["adc_info"].set_index("planet_id")["AIRS-CH0_adc_offset"].loc[int(id)]
        )
        signal = self._adc_revert(signal, gain, offset)
        dt = info["axis_info"]["AIRS-CH0-integration_time"].dropna().values
        dt[1::2] += 0.1
        signal = signal[:, :, self.CUT_INF : self.CUT_SUP]
        if self.MASK:
            signal = self._mask_hot_dead(signal, data["dead"], data["dark"])
        if self.CORR:
            signal = self._apply_linear_corr(data["linear_corr"], signal)
        if self.DARK:
            signal = self._clean_dark_current(signal, data["dead"], data["dark"], dt)
        signal = self._get_cds(signal)
        if self.BINNING:
            signal = self._bin_obs(signal, self.BINNING)
        else:
            signal = signal.transpose(0, 2, 1)
        if self.FLAT:
            signal = self._correct_flat_field(data["flat"], data["dead"], signal)
        return signal

    def _calibrate_fgs_data(self, data: dict, info: dict):
        signal = data["signal"]
        gain = info["adc_info"].set_index("planet_id")["FGS1_adc_gain"].loc[int(id)]
        offset = info["adc_info"].set_index("planet_id")["FGS1_adc_offset"].loc[int(id)]
        signal = self._adc_revert(signal, gain, offset)
        dt = np.ones(len(signal)) * 0.1
        dt[1::2] += 0.1
        signal = signal[:, :, self.CUT_INF : self.CUT_SUP]
        if self.MASK:
            signal = self._mask_hot_dead(signal, data["dead"], data["dark"])
        if self.CORR:
            signal = self._apply_linear_corr(data["linear_corr"], signal)
        if self.DARK:
            signal = self._clean_dark_current(signal, data["dead"], data["dark"], dt)
        signal = self._get_cds(signal)
        if self.BINNING:
            signal = self._bin_obs(signal, self.BINNING * 10)
        else:
            signal = signal.transpose(0, 2, 1)
        if self.FLAT:
            signal = self._correct_flat_field(data["flat"], data["dead"], signal)
        return signal

    def _adc_revert(signal: np.ndarray, gain: float, offset: float) -> np.ndarray:
        """Revert pixel voltage from ADC.

        Args:
            signal (np.ndarray): ADC converted signal integer.
            gain (float): ADC gain error.
            offset (float): ADC offset error.

        Returns:
            np.ndarray: Pixel voltages.
        """
        signal = signal.astype(np.float64)
        signal /= gain
        signal += offset
        return signal

    def _mask_hot_dead(
        self, signal: np.ndarray, dead: np.ndarray, dark: np.ndarray
    ) -> np.ndarray:
        """Mask dead and hot pixels so that they won't be take in account in corrections.

        Args:
            signal (np.ndarray): Pixel voltage signal.
            dead (np.ndarray): Dead pixels.
            dark (np.ndarray): Dark pixels.

        Returns:
            np.ndarray: Pixel voltages with dead pixels masked.
        """
        hot = sigma_clip(dark, sigma=5, maxiters=5).mask
        hot = np.tile(hot, (signal.shape[0], 1, 1))
        dead = np.tile(dead, (signal.shape[0], 1, 1))
        signal = np.ma.masked_where(dead, signal)
        signal = np.ma.masked_where(hot, signal)
        return signal

    def _apply_linear_corr(self, corr: np.ndarray, signal: np.ndarray) -> np.ndarray:
        """Fix non-linearity due to capacity leakage in the detector.

        Args:
            corr (np.ndarray): Correction coefficients
            signal (np.ndarray): Signal to correct.

        Returns:
            np.ndarray: Corrected signal
        """
        linear_corr = np.flip(corr, axis=0)
        for x, y in itertools.product(range(signal.shape[1]), range(signal.shape[2])):
            poli = np.poly1d(linear_corr[:, x, y])
            signal[:, x, y] = poli(signal[:, x, y])
        return signal

    def _clean_dark_current(
        self, signal: np.ndarray, dead: np.ndarray, dark: np.ndarray, dt: np.array
    ) -> np.ndarray:
        """Remove the accumulated charge due to dark current.

        Args:
            signal (np.ndarray): Signal to clean
            dead (np.ndarray): Dead pixels.
            dark (np.ndarray): Dark pixels.
            dt (np.array): Short frames delay.

        Returns:
            np.ndarray: Cleaned signal.
        """
        dark = np.ma.masked_where(dead, dark)
        dark = np.tile(dark, (signal.shape[0], 1, 1))

        signal -= dark * dt[:, np.newaxis, np.newaxis]
        return signal

    def _get_cds(self, signal: np.ndarray) -> np.ndarray:
        """Return the actual accumulated charge (a delta) due to the transit.

        Args:
            signal (np.ndarray): Signal.

        Returns:
            np.ndarray: An image for one observation in the time (Time series observations).
        """
        cds = signal[1::2, :, :] - signal[::2, :, :]
        return cds

    def _bin_obs(self, cds: np.ndarray, binning: int) -> np.ndarray:
        """Binnes cds time series together at the specified frequency.

        Args:
            cds (np.ndarray): CDS signal.
            binning (int): Binning frequency.

        Returns:
            np.ndarray: _description_
        """
        cds_transposed = cds.transpose(0, 2, 1)
        cds_binned = np.zeros(
            (
                cds_transposed.shape[0] // binning,
                cds_transposed.shape[1],
                cds_transposed.shape[2],
            )
        )
        for i in range(cds_transposed.shape[0] // binning):
            cds_binned[i, :, :] = np.sum(
                cds_transposed[i * binning : (i + 1) * binning, :, :], axis=0
            )
        return cds_binned

    def _correct_flat_field(
        self, flat: np.ndarray, dead: np.ndarray, signal: np.ndarray
    ):
        """Correction by calibrating on an uniform signal.

        Args:
            flat (np.ndarray): Flat signal
            dead (np.ndarray): Dead pixels
            signal (np.ndarray): CDS signal
        """

        flat = flat.transpose(1, 0)
        dead = dead.transpose(1, 0)
        flat = np.ma.masked_where(dead, flat)
        flat = np.tile(flat, (signal.shape[0], 1, 1))
        signal = signal / flat
        return signal

    def _load_data(self, uris: List[str]):
        """
        Load data from a list of URIs, reading the files as CSV or Parquet,
        and organize them by type ('dark', 'dead', etc.) for AIRS and FGS.

        Args:
            uris (List[str]): List of URIs to load data from.
        """
        airs_data = {}
        fgs_data = {}
        info_data = {}
        calib_data_types = [
            "dark",
            "dead",
            "flat",
            "linear_corr",
            "read",
            "signal",
            "axis_info",
            "test_adc_info",
            "train_adc_info",
        ]

        def read_data(uri: str):
            try:
                return pd.read_csv(uri)
            except UnicodeDecodeError:
                return pd.read_parquet(uri)

        for uri in uris:
            df = read_data(uri)
            for data_type in calib_data_types:
                if data_type in uri:
                    if "AIRS" in uri:
                        if "signal" in uri:
                            airs_data[data_type] = df.values.astype(np.float64).reshape(
                                (df.shape[0], 32, 356)
                            )
                        elif "linear_corr" in uri:
                            airs_data[data_type] = df.values.astype(np.float64).reshape(
                                (6, 32, 356)
                            )[:, :, self.CUT_INF : self.CUT_SUP]
                        else:
                            airs_data[data_type] = df.values.astype(np.float64).reshape(
                                (32, 356)
                            )[:, self.CUT_INF : self.CUT_SUP]
                    elif "FGS" in uri:
                        if "signal" in uri:
                            fgs_data[data_type] = df.values.astype(np.float64).reshape(
                                (df.shape[0], 32, 32)
                            )
                        elif "linear_corr" in uri:
                            fgs_data[data_type] = df.values.astype(np.float64).reshape(
                                (6, 32, 32)
                            )[:, :, self.CUT_INF : self.CUT_SUP]
                        else:
                            fgs_data[data_type] = df.values.astype(np.float64).reshape(
                                (32, 32)
                            )[:, self.CUT_INF : self.CUT_SUP]
                    else:
                        info_data[data_type] = df
        info_data["adc_info"] = pd.concat(
            [info_data["train_adc_info"], info_data["test_adc_info"]]
        )
        del info_data["train_adc_info"], info_data["test_adc_info"]
        return airs_data, fgs_data, info_data

In [11]:
processor = CalibrationFn(39, 321, 30, True, True, True, True)

In [7]:
processor.CUT_INF

39

In [28]:
df = pd.read_csv("/home/drxc/neurips_adc/data/raw/test_adc_info.csv")
df.set_index("planet_id")["FGS1_adc_offset"].loc[499191466]

-331.0330086611678

In [27]:
df

FGS1_adc_offset       -331.033009
FGS1_adc_gain            0.823442
AIRS-CH0_adc_offset   -537.692100
AIRS-CH0_adc_gain        0.937944
star                     1.000000
Name: 499191466, dtype: float64

In [13]:
result = processor.process(
    (
        "499191466",
        [
            "gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/dark.parquet",
            "gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/dead.parquet",
            "gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/flat.parquet",
            "gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/linear_corr.parquet",
            "gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/read.parquet",
            "gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_signal.parquet",
            "gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/dark.parquet",
            "gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/dead.parquet",
            "gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/flat.parquet",
            "gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/linear_corr.parquet",
            "gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/read.parquet",
            "gs://neurips-adc-bucket/raw/test/499191466/FGS1_signal.parquet",
            "gs://neurips-adc-bucket/raw/axis_info.parquet",
            "gs://neurips-adc-bucket/raw/sample_submission.csv",
            "gs://neurips-adc-bucket/raw/test_adc_info.csv",
            "gs://neurips-adc-bucket/raw/train_adc_info.csv",
            "gs://neurips-adc-bucket/raw/train_labels.csv",
            "gs://neurips-adc-bucket/raw/wavelengths.csv",
        ],
    )
)

KeyError: 499191466

In [93]:
with beam.Pipeline() as pipeline:
    uris = get_raw_data_uris("gs://neurips-adc-bucket/**")
    outputs = pipeline | "Create initial values" >> beam.Create(uris)
    outputs | beam.Map(print)

('499191466', ['gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/dark.parquet', 'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/dead.parquet', 'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/flat.parquet', 'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/linear_corr.parquet', 'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_calibration/read.parquet', 'gs://neurips-adc-bucket/raw/test/499191466/AIRS-CH0_signal.parquet', 'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/dark.parquet', 'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/dead.parquet', 'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/flat.parquet', 'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/linear_corr.parquet', 'gs://neurips-adc-bucket/raw/test/499191466/FGS1_calibration/read.parquet', 'gs://neurips-adc-bucket/raw/test/499191466/FGS1_signal.parquet', 'gs://neurips-adc-bucket/raw/axis_info.parquet', 'gs://